# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы:
- настроите метод опорных векторов
- изучите методы работы с категориальными переменными

In [27]:
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder as OHE
import copy
from joblib import Parallel, delayed
import warnings, time
import numpy as np
warnings.simplefilter(action='ignore')

__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(5 баллов)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [28]:
data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [29]:
np.random.seed(42)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)
mask = np.concatenate([mask_plus, mask_zero])
mask = np.sort(mask)

data = data.iloc[mask]
target = target[mask]

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5, random_state=42)

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [30]:
def scall(data):
    new_data = copy.deepcopy(data)
    scaler = StandardScaler()
    new_data[new_data.select_dtypes(include=['float64', 'int64']).columns] = scaler.fit_transform(new_data.select_dtypes(include=['float64', 'int64']))

    return new_data


Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию. Сделайте предсказание на тестовой части выборки. Посчитайте accuracy, precision, recall и F меру

In [31]:
def train(X_train, X_test, y_train, y_test):
    st = time.time()
    model = LogisticRegression(max_iter=2000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy, precision, recall, f1 = accuracy_score(y_test, y_pred) , precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred)
    print(f"Accuracy:", accuracy)
    print(f"Precision:", precision)
    print(f"Recall:", recall)
    print(f"F1 Score:", f1)
    en = time.time()
    print(f"Time taken: {en - st} s")


train(*train_test_split(scall(data), target, test_size=0.5, random_state=42))


Accuracy: 0.59124
Precision: 0.5975506665221633
Recall: 0.5527196904384787
F1 Score: 0.5742615506395033
Time taken: 0.5159845352172852 s


__Выводы__ в свободной форме: Ну дисбаланса классов у нас особо нет, судя по F1, но... У нас само по себе маленькое качество и предсказание почти не отличается от случайного. Для людей, ушедших далеко вперёд (мы вернулись к этому заданию спустя примерно полгода) это ужасное качество

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [32]:
data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.984535,1.366685,4.475825,0.427785,0.498775,0.354095,0.293855,0.175180,0.17687,0.000485,0.001815,0.011305,0.001190,0.014795,7.097650,0.628765,0.150620,0.157775,0.625666,0.470882,0.607556,8.393725,0.801825,-0.442505,0.894515,-0.114750,6.679825,0.879415,0.813875,1.342500,0.992540,62.632245,2.344055,0.385316,0.841750,0.272450,3.113360,0.450418,0.450681,0.451150,2.371435,1.890125,7.689680,3.005820,9.223045,2.341310,8.448270,5.449475,1.441545,2.872185,7.549840,0.123410,0.628155,0.553315,0.287320,0.345090,0.153685
std,2.013988,0.674692,2.749082,0.496383,1.497825,0.478239,0.455527,0.380122,0.38156,0.022017,0.042564,0.105723,0.034476,0.140983,3.563262,0.483136,0.357679,0.364531,0.283914,0.423068,0.780740,2.563911,0.398638,0.821375,2.381902,0.843212,5.523387,0.406577,0.389208,0.972316,0.089467,33.218457,0.840711,0.061116,0.244495,0.365705,0.694621,0.286845,0.285969,0.286302,1.115503,1.136679,1.329891,1.414704,1.461628,1.245149,2.904012,2.340659,1.206416,1.690382,2.749935,0.328908,0.483299,0.497151,0.452513,0.475399,0.360647
min,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,1.000000,-1.000000,0.141421,0.309258,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.400000,0.200000,0.551135,7.000000,1.000000,-1.000000,0.000000,-1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,32.000000,2.000000,0.316228,0.685113,0.330303,2.828427,0.200000,0.200000,0.200000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.700000,0.300000,0.753741,8.000000,1.000000,-1.000000,0.000000,0.000000,7.000000,1.000000,1.000000,2.000000,1.000000,65.000000,3.000000,0.387298,0.787037,0.368782,3.316625,0.500000,0.500000,0.500000,2.000000,2.000000,8.000000,3.0000

Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [33]:
categorical_features = [c for c in data.columns if c.endswith('_cat')]

encoder = OHE(sparse_output=False, drop="first")
encoded_features = encoder.fit_transform(data[categorical_features])

data_one_hot = data.drop(categorical_features, axis=1)
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

data_one_hot.reset_index(inplace=True, drop=True)
data_one_hot = pd.concat([data_one_hot, encoded_df], axis=1)

train(*train_test_split(scall(data_one_hot), target, test_size=0.5, random_state=73))

Accuracy: 0.59458
Precision: 0.6022219825261569
Recall: 0.5581079189907835
F1 Score: 0.5793263743333265
Time taken: 2.7253832817077637 s


__Вывод:__ Качество выросло немножко. Из своего маленького опыта могу сказать, что OHE не всегда применять оправданно. Как минимум это усложняет анализ и интерпретацию датасета (ну можно тогда его применить после). Ну очевидно есть и плюсы. Именно они дали прирост

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

По объектам выше в датасете

In [34]:
X = data.copy()
y = target.copy()
for feature in categorical_features:
    ind = 0
    ans = []
    mp = {}
    for j in X[feature].tolist():
        mp[j] = mp.get(j, [0, 0])
        mp[j][0] += y[ind]
        mp[j][1] += 1
        ans.append(mp[j][0] / mp[j][1])
        ind += 1
    X[feature] = pd.Series(ans, index=X[feature].index)

train(*train_test_split(scall(X), y, test_size=0.5, random_state=42))

Accuracy: 0.5972
Precision: 0.6017558368852981
Recall: 0.5689395913948313
F1 Score: 0.5848877712966589
Time taken: 0.5351383686065674 s


По фолдам

In [35]:
X = data.copy()
y = target.copy()
B = int(len(X) ** 0.5)

for feature in categorical_features:
    lst = X[feature].to_list()
    cnts = [{} for j in range(len(X) // B + 2)]
    sums = [{} for j in range(len(X) // B + 2)]

    for i in range(len(X)):
        m = i // B
        if len(cnts[m]) == 0:
            cnts[m + 1] = cnts[m].copy()
            sums[m + 1] = sums[m].copy()
        cnts[m + 1][lst[i]] = cnts[m + 1].get(lst[i], 0) + 1
        sums[m + 1][lst[i]] = sums[m + 1].get(lst[i], 0) + y[i]

    ind = 0
    ans = []
    for j in X[feature].tolist():
        m = ind// B
        sum = sums[m].get(j, 0) + sums[-1].get(j, 0) - sums[m + 1].get(j, 0)
        cnt = cnts[m].get(j, 0) + cnts[-1].get(j, 0) - cnts[m + 1].get(j, 0)
        ans.append(0 if cnt == 0 else sum / cnt)
        ind += 1

    X[feature] = pd.Series(ans, index=X[feature].index)

train(*train_test_split(scall(X), y, test_size=0.5, random_state=42))

Accuracy: 0.6138
Precision: 0.6178373285878781
Recall: 0.5916755217835876
F1 Score: 0.6044734847708978
Time taken: 0.4910874366760254 s


__Вывод:__ От фолдов мы получили прирост качества примерно на 2 процента. Это уже хоть что-то, но, возможно, было бы лучше использовать другую модель для обучения...